<h1>Покупка квартиры мечты</h1>
<img src="media/dream_apartment.jpg/">

Вводные:
1. Покупаем квартирку.
2. Будет ипотека. Ограничения: первоначальный взнос 1.8 млн рублей, 5 лет, платеж до 200 тыр.


<span style="color:#ffffff; background: #0703fc">Цель исследования: сравнить рост цены недвижки и переплату по ипотеке</span>

План анализа:
1. Изучить расчет платежей по ипотеке. Аннуитетные платежи. Досрочные платежи.
2. Выбрать и распарсить данные по недвиге - собрать датасет.
4. Найти старый парсинг циана, посмотреть, как растут в цене новостройки по районам и застройщикам.
5. Посчитать что выгоднее.

<h2>Функции для расчета ипотеки</h2>

In [1]:
import pandas as pd

# считаем платеж и переплату при аннуитетном платеже
# нужно передать датафрейм с полями
# стоимость хаты (s), первоначальный взнос (fst), срок ипотеки в месяцах (n), годовая ставка (p)
def annuit(str):
    p = str.p / 12
    debt = str.s - str.fst
    payment = debt * (p + p / (pow((1 + p), str.n) - 1))
    overpay = payment * str.n - debt
    return payment, overpay

# возвращает график платежей
# нужно передать датафрейм с полями
# стоимость хаты (s), первоначальный взнос (fst), срок ипотеки в месяцах (n), годовая ставка (p)
def payment_schedule(str):
    payments_list = []
    rest = str.s - str.fst
    n = int(str.n)
    i = 1
    while (i <= n) & (rest > 0) :
        perc_mon = rest * str.p/12
        real_payment = str.payment
        if real_payment > rest:
            real_payment = rest + perc_mon
        debt_mon = real_payment - perc_mon
        rest = rest - debt_mon
        payments_list.append([i, real_payment, rest, perc_mon, debt_mon])
        # print(i, real_payment, rest, perc_mon, debt_mon)
        i+=1
    df = pd.DataFrame(payments_list)
    df = df.rename(columns={0:'i', 1:'payment', 2:'rest', 3:'perc_mon', 4:'debt_mon'})
    return df

# возвращает график платежей с досрочным погашением
# нужно передать датафрейм с полями
# стоимость хаты (s), первоначальный взнос (fst), срок ипотеки в месяцах (n), годовая ставка (p)
# и список досрочных погашений с полями [[0, 1, 2, 3]]
# сколько платить (0), периодичность (1) 0 - разово, 1 - раз в месяц, 2 - раз в 2 месяца и тд, начало (2), конец (3)
# можно несколько досрочных погашений
def payment_schedule_early(str, eps):
    payments_list = []
    rest = str.s - str.fst
    n = int(str.n)
    i = 1
    while (i <= n) & (rest > 0) :
        perc_mon = rest * str.p/12
        real_payment = str.payment
        for ep in eps:
            if i in range(ep[2], ep[3]):
                if (ep[1] == 0)|((i - ep[2]) % ep[1] == 0):
                    real_payment = ep[0]
        if real_payment > rest:
            real_payment = rest + perc_mon
        debt_mon = real_payment - perc_mon
        rest = rest - debt_mon
        payments_list.append([i, real_payment, rest, perc_mon, debt_mon])
        i+=1
    df = pd.DataFrame(payments_list)
    df = df.rename(columns={0:'i', 1:'payment', 2:'rest', 3:'perc_mon', 4:'debt_mon'})
    return df

Посчитаем переплату по желаемой хате с разными параметрами

In [3]:
10361000+1828868

12189868

In [2]:
# стоимость хаты, первоначальный взнос, срок ипотеки в месяцах, годовая ставка
params = [(12189868, 1828868, 60, 0.044),
          (12189868, 1828868, 240, 0.044),
          (12189868, 1828868, 360, 0.044)]

In [3]:
import pandas as pd

diff = pd.DataFrame(params)
diff = diff.rename(columns={0:'s', 1:'fst', 2:'n', 3:'p'})
diff['debt'] = diff['s'] - diff['fst']
diff[['payment', 'overpay']] =  diff.apply(lambda row: annuit(row), axis='columns', result_type='expand')
diff.style.format({'fst':'{:.2f}','p': '{:.3f}', 'debt':'{0:_}', 'payment':'{0:_.2f}', 'overpay':'{0:_.2f}'})
diff.head()

,s,fst,n,p,debt,payment,overpay
0,12189868,1828868,60,0.044,10361000,192689.550538,1.200373e+06
1,12189868,1828868,240,0.044,10361000,64990.844899,5.236803e+06
2,12189868,1828868,360,0.044,10361000,51883.837906,8.317182e+06


Кредит дадут на миллиард лет, но гасить лучше досрочно. Посчитаем, лучше сразу ебашить по 200 тыр или можно начать через 3 месяца.

In [4]:
# сколько платить, периодичность (0 - разово, 1 - раз в месяц, 2 - раз в 2 месяца и тд), начало, конец
early_payments =[[200000, 1, 4, 10000]]
payments = payment_schedule_early(diff.loc[2],early_payments)
overpay1 = payments.perc_mon.sum()
print('Overpay = ', overpay1)
print('n = ',payments.shape[0])

# сколько платить, периодичность (0 - разово, 1 - раз в месяц, 2 - раз в 2 месяца и тд), начало, конец
early_payments =[[200000, 1, 1, 10000]]
payments = payment_schedule_early(diff.loc[2],early_payments)
overpay2 = payments.perc_mon.sum()
print('Overpay = ', overpay2)
print('n = ',payments.shape[0])
print(overpay1-overpay2)

Overpay =  1254688.015664503
n =  61
Overpay =  1150763.2944132632
n =  58
103924.72125123977


ПЕРЕПЛАТА БУДЕТ СТОИТЬ 100 ТЫЩ, ГАСИМ СРАЗУ

In [7]:
# Посмотрим, сколько надо будет гасить кредит, если вкидывать по 200к ежемесячно
early_payments =[[200000, 1, 1, 10000]]
payments = payment_schedule_early(diff.loc[2],early_payments)
payments

,i,payment,rest,perc_mon,debt_mon
0,1,200000.000000,1.019899e+07,37990.333333,162009.666667
1,2,200000.000000,1.003639e+07,37396.297889,162603.702111
2,3,200000.000000,9.873187e+06,36800.084314,163199.915686
3,4,200000.000000,9.709388e+06,36201.684624,163798.315376
4,5,200000.000000,9.544989e+06,35601.090801,164398.909199
5,6,200000.000000,9.379988e+06,34998.294800,165001.705200
6,7,200000.000000,9.214381e+06,34393.288548,165606.711452
7,8,200000.000000,9.048167e+06,33786.063939,166213.936061
8,9,200000.000000,8.881344e+06,33176.612840,166823.387160
9,10,200000.000000,8.713909e+06,32564.927087,167435.072913


In [8]:
# переплата
payments['perc_mon'].sum()

1150763.2944132632

In [9]:
# Посмотрим, сколько надо будет гасить кредит, если вкидывать по 250к ежемесячно
early_payments =[[250000, 1, 1, 10000]]
payments = payment_schedule_early(diff.loc[2],early_payments)
payments

,i,payment,rest,perc_mon,debt_mon
0,1,250000.000000,1.014899e+07,37990.333333,212009.666667
1,2,250000.000000,9.936203e+06,37212.964556,212787.035444
2,3,250000.000000,9.722636e+06,36432.745426,213567.254574
3,4,250000.000000,9.508286e+06,35649.665492,214350.334508
4,5,250000.000000,9.293149e+06,34863.714266,215136.285734
5,6,250000.000000,9.077224e+06,34074.881218,215925.118782
6,7,250000.000000,8.860507e+06,33283.155782,216716.844218
7,8,250000.000000,8.642996e+06,32488.527354,217511.472646
8,9,250000.000000,8.424687e+06,31690.985287,218309.014713
9,10,250000.000000,8.205577e+06,30890.518900,219109.481100


In [10]:
payments['perc_mon'].sum()

897950.9347436822

<h2>Парсим сайты с недвигой</h2>

In [1]:
import requests
from bs4 import BeautifulSoup # for web scraping
import pandas as pd
import numpy as np
import re
from ipywidgets import IntProgress #for progress bar
from IPython.display import display

In [70]:
req = requests.get('https://m2.ru/sankt-peterburg/nedvizhimost/kupit-kvartiru/')
soup = BeautifulSoup(req.text, "lxml")

In [71]:
all_hrefs = [a.get('href') for a in soup.find_all('a')] #list of all links
flat_hrefs=[a for a in all_hrefs if 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha' in str(a)] #filtering the list
flat_hrefs=list(set(flat_hrefs)) #removing duplicates in the list
flat_hrefs[0:5]

['https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-1-komnatnoi-kvartiri-53-3-m-1-6-etazh-672d9b9c-8827-42ca-b7be-736115ce772e/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-1-komnatnoi-kvartiri-37-1-m-3-9-etazh-c886fe91-b6cb-4533-b7ba-688773fd11fc/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-2-komnatnoi-kvartiri-67-4-m-6-12-etazh-6ac17150-d91b-4273-af15-f659059d7901/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-2-komnatnoi-kvartiri-63-1-m-8-8-etazh-d9d2f232-05ad-48d2-819e-1bfdd7e4e12a/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-1-komnatnoi-kvartiri-33-9-m-8-14-etazh-bf400424-6e48-43de-be62-ea1e6ba5b4e7/']

In [72]:
def add_flats(hrefs):
    flats = pd.DataFrame()
    progressbar=IntProgress(min=0, max=len(hrefs), value=0) #progress bar
    display(progressbar)
    for i,href in enumerate(hrefs):
        # getting html code of the flat page
        try:
            req1 = requests.get(href)
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        soup = BeautifulSoup(req1.text, "lxml")
        flat = {'Ссылка': href,
               'Название': soup.h1.text.replace('\xa0',' '),
               'Адрес': ', '.join(a.string for a in soup.find_all(attrs={"class": "ClClickableAddress__link"})),
               'Цена': soup.find(attrs={"itemprop": "price"}).text.replace('\xa0',' ')}
        attr = soup.find_all(attrs={"class":"colors-named-module__secondary___2WPH_ fonts-module__secondary-alone___3wVR-"})
        vals = soup.find_all(attrs={"class":"fonts-module__primary___2PNSt DescriptionCell__value"})
        for a,v in zip(attr,vals):
            flat[a.text] = v.text.replace('\xa0',' ')
            progressbar.value = i+1
        flats = pd.concat([flats, pd.DataFrame([flat])], ignore_index=True)
    return flats

In [77]:
flats_all = pd.DataFrame()

In [ ]:
n_page=373
while n_page<=500:
    print('page number:',str(n_page))
    try:
        req = requests.get('https://m2.ru/sankt-peterburg/nedvizhimost/kupit-kvartiru/?pageNumber='+str(n_page))
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    soup = BeautifulSoup(req.text, "lxml")
    # if we don't have header we stop parsing
    if soup.h1 is None:
        print('Parsing is stopped')
        break
    #getting all links from the page
    all_hrefs = [a.get('href') for a in soup.find_all('a')]
    flat_hrefs = [a for a in all_hrefs if 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha' in str(a)]
    #getting all parameters from an apartment page.
    flat_hrefs = list(set(flat_hrefs))
    flats = add_flats(flat_hrefs)
    flats_all = pd.concat([flats_all, flats], ignore_index=True)
    print('number of rows',flats_all.shape[0])
    n_page+=1

In [131]:
flats_all

,Ссылка,Название,Адрес,Цена,Комнат,Отделка,Площадь квартиры,Жилая площадь,Площадь кухни,Этаж,...,Кухонная мебель,Телевизор,Стиральная машина,Год постройки,Консьерж,Материал стен,Отопление,Вид из окон,Серия дома,Временная эпоха
0,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 1-комнатной квартиры 34 м², 9/12 этаж","Санкт-Петербург, Выборгский район, Большой Сам...",10 500 000,1,чистовая,34 м²,"16,2 м²","10,1 м²",9 из 12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 1-комнатной квартиры 53,3 м², 1/6 этаж","Санкт-Петербург, Петроградский район, Петровск...",32 700 000,1,NaN,"53,3 м²",NaN,NaN,1 из 6,...,есть,есть,есть,2018,есть,NaN,NaN,NaN,NaN,NaN
2,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 1-комнатной квартиры 37,1 м², 3/9 этаж","Санкт-Петербург, Невский район, проспект Обухо...",11 750 289,1,чистовая,"37,1 м²","17,3 м²","10,1 м²",3 из 9,...,NaN,NaN,NaN,NaN,NaN,кирпич-монолит,NaN,NaN,NaN,NaN
3,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 2-комнатной квартиры 67,4 м², 6/12 этаж","Санкт-Петербург, Красносельский район, Ленинск...",12 500 000,2,NaN,"67,4 м²","34,4 м²","15,4 м²",6 из 12,...,есть,NaN,есть,NaN,есть,монолит,центральное,NaN,NaN,NaN
4,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 2-комнатной квартиры 63,1 м², 8/8 этаж","Санкт-Петербург, Московский район, Черниговска...",17 490 000,2,NaN,"63,1 м²","33,2 м²","10,2 м²",8 из 8,...,NaN,NaN,NaN,2023,NaN,кирпич-монолит,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа квартиры-студии 30,2 м², 5/23 этаж","Санкт-Петербург, Выборгский район, Парголово, ...",6 447 519,NaN,предчистовая,"30,2 м²","19,1 м²",NaN,5 из 23,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,NaN
9996,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа квартиры-студии 33,1 м², 15/25 этаж","Санкт-Петербург, Приморский район, Богатырский...",8 400 000,NaN,NaN,"33,1 м²","21,4 м²",NaN,15 из 25,...,NaN,NaN,NaN,2022,NaN,кирпич-монолит,NaN,NaN,NaN,NaN
9997,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа квартиры-студии 30,4 м², 4/6 этаж","Санкт-Петербург, Московский район, улица Коли ...",7 476 000,NaN,NaN,"30,4 м²","17,9 м²",NaN,4 из 6,...,NaN,NaN,NaN,NaN,NaN,кирпич,NaN,NaN,NaN,NaN
9998,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 2-комнатной квартиры 45 м², 6/9 этаж","Санкт-Петербург, Выборгский район, Выборгское ...",6 600 000,2,NaN,45 м²,"27,5 м²","6,4 м²",6 из 9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
flats_all.to_csv('data/raw_data_from_metr2_2023.csv',sep=';',index=False)

<h2>Почистим данные</h2>

In [23]:
import pandas as pd
flats = pd.read_csv('data/raw_data_from_metr2_2023.csv',sep=';')

In [24]:
flats.drop_duplicates(ignore_index=True,inplace=True)
flats.shape

(9193, 36)

In [25]:
flats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9193 entries, 0 to 9192
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ссылка             9193 non-null   object 
 1   Название           9193 non-null   object 
 2   Адрес              9193 non-null   object 
 3   Цена               9193 non-null   object 
 4   Комнат             7517 non-null   float64
 5   Отделка            3732 non-null   object 
 6   Площадь квартиры   9193 non-null   object 
 7   Жилая площадь      8794 non-null   object 
 8   Площадь кухни      6985 non-null   object 
 9   Этаж               9192 non-null   object 
 10  Санузел            6600 non-null   object 
 11  Кол-во санузлов    6909 non-null   float64
 12  Балкон/лоджия      6349 non-null   object 
 13  Вид из окна        3919 non-null   object 
 14  Срок сдачи         3889 non-null   object 
 15  Высота потолков    6697 non-null   object 
 16  Лифт               5704 

In [26]:
flats['Адрес'].head()

0    Санкт-Петербург, Выборгский район, Большой Сам...
1    Санкт-Петербург, Петроградский район, Петровск...
2    Санкт-Петербург, Невский район, проспект Обухо...
3    Санкт-Петербург, Красносельский район, Ленинск...
4    Санкт-Петербург, Московский район, Черниговска...
Name: Адрес, dtype: object

In [27]:
# попробуем split
flats['Адрес'].str.split(',', expand=True)

,0,1,2,3,4,5
0,Санкт-Петербург,Выборгский район,Большой Сампсониевский проспект,7,None,None
1,Санкт-Петербург,Петроградский район,Петровский проспект,2с2,None,None
2,Санкт-Петербург,Невский район,проспект Обуховской Обороны,None,None,None
3,Санкт-Петербург,Красносельский район,Ленинский проспект,72к1,None,None
4,Санкт-Петербург,Московский район,Черниговская улица,11к1,None,None
...,...,...,...,...,...,...
9188,Санкт-Петербург,Адмиралтейский район,Спасский переулок,6-8,None,None
9189,Санкт-Петербург,Фрунзенский район,Бухарестская улица,72к1,None,None
9190,Санкт-Петербург,Выборгский район,Парголово,Ольгинская дорога,3/20,None
9191,Санкт-Петербург,Приморский район,Богатырский проспект,2А,None,None


Как раскидать адрес:
- первая колонка - всегда город
- вторая колонка - всегда район
- улицы и номера дома может не быть
- улица может быть в последней заполненной колонке или в колонке до номера дома
- номер дома всегда последний и содержит цифры
- все остальное не нужно

In [28]:
import re
temp_addr = flats['Адрес'].str.split(',')
flats['Город'] = [a[0] for a in temp_addr]
flats['Район'] = [a[1] for a in temp_addr]
temp_addr = [a[2:] for a in temp_addr]
flats['Улица'] = ['' if len(a)==0 else a[0] if len(a)==1 else a[-2] for a in temp_addr]
flats['Номер дома'] = [a[-1] if len(a)>0 else '' for a in temp_addr]
flats['Номер дома'] = flats['Номер дома'].apply(lambda a:a if bool(re.search(r'\d',a)) else '')

In [29]:
flats[['Адрес', 'Город', 'Район', 'Улица', 'Номер дома']]

,Адрес,Город,Район,Улица,Номер дома
0,"Санкт-Петербург, Выборгский район, Большой Сам...",Санкт-Петербург,Выборгский район,Большой Сампсониевский проспект,7
1,"Санкт-Петербург, Петроградский район, Петровск...",Санкт-Петербург,Петроградский район,Петровский проспект,2с2
2,"Санкт-Петербург, Невский район, проспект Обухо...",Санкт-Петербург,Невский район,проспект Обуховской Обороны,
3,"Санкт-Петербург, Красносельский район, Ленинск...",Санкт-Петербург,Красносельский район,Ленинский проспект,72к1
4,"Санкт-Петербург, Московский район, Черниговска...",Санкт-Петербург,Московский район,Черниговская улица,11к1
...,...,...,...,...,...
9188,"Санкт-Петербург, Адмиралтейский район, Спасски...",Санкт-Петербург,Адмиралтейский район,Спасский переулок,6-8
9189,"Санкт-Петербург, Фрунзенский район, Бухарестск...",Санкт-Петербург,Фрунзенский район,Бухарестская улица,72к1
9190,"Санкт-Петербург, Выборгский район, Парголово, ...",Санкт-Петербург,Выборгский район,Ольгинская дорога,3/20
9191,"Санкт-Петербург, Приморский район, Богатырский...",Санкт-Петербург,Приморский район,Богатырский проспект,2А


In [30]:
flats['Цена'].head()

0    10 500 000
1    32 700 000
2    11 750 289
3    12 500 000
4    17 490 000
Name: Цена, dtype: object

In [31]:
flats['Цена'] = flats['Цена'].str.replace(' ','').astype(float)
flats['Цена'].head()

0    10500000.0
1    32700000.0
2    11750289.0
3    12500000.0
4    17490000.0
Name: Цена, dtype: float64

In [32]:
flats['Комнат'].head()

0    1.0
1    1.0
2    1.0
3    2.0
4    2.0
Name: Комнат, dtype: float64

In [33]:
# 0 комнат - это похоже студия
flats['Комнат'] = flats['Комнат'].fillna(0).astype(int)
flats['Комнат'].value_counts()

Комнат
1     3558
2     2390
0     1676
3     1310
4      183
5       44
7       13
6       12
8        3
11       1
10       1
13       1
9        1
Name: count, dtype: int64

In [34]:
flats['Отделка'].head()

0    чистовая
1         NaN
2    чистовая
3         NaN
4         NaN
Name: Отделка, dtype: object

In [35]:
flats['Отделка'] = flats['Отделка'].fillna('')
flats['Отделка'].value_counts()

Отделка
                5461
чистовая        2634
без отделки      927
предчистовая     171
Name: count, dtype: int64

In [36]:
flats['Площадь квартиры'].head()

0      34 м²
1    53,3 м²
2    37,1 м²
3    67,4 м²
4    63,1 м²
Name: Площадь квартиры, dtype: object

In [37]:
# Добавим сразу цену за метр
flats['Площадь квартиры'] = flats['Площадь квартиры'].str.replace('м²','').str.replace(',','.').astype(float)
flats['Цена за метр'] = flats['Цена'] / flats['Площадь квартиры']
flats[['Цена', 'Площадь квартиры', 'Цена за метр']].head()

,Цена,Площадь квартиры,Цена за метр
0,10500000.0,34.0,308823.529412
1,32700000.0,53.3,613508.442777
2,11750289.0,37.1,316719.380054
3,12500000.0,67.4,185459.940653
4,17490000.0,63.1,277179.080824


In [38]:
flats[['Жилая площадь','Площадь кухни']].head()

,Жилая площадь,Площадь кухни
0,"16,2 м²","10,1 м²"
1,NaN,NaN
2,"17,3 м²","10,1 м²"
3,"34,4 м²","15,4 м²"
4,"33,2 м²","10,2 м²"


In [39]:
flats['Жилая площадь'] = flats['Жилая площадь'].str.replace('м²','').str.replace(',','.').fillna(0).astype(float)
flats['Площадь кухни'] = flats['Площадь кухни'].str.replace('м²','').str.replace(',','.').fillna(0).astype(float)
flats[['Жилая площадь','Площадь кухни']].head()

,Жилая площадь,Площадь кухни
0,16.2,10.1
1,0.0,0.0
2,17.3,10.1
3,34.4,15.4
4,33.2,10.2


In [40]:
flats['Этаж'].head()

0    9 из 12
1     1 из 6
2     3 из 9
3    6 из 12
4     8 из 8
Name: Этаж, dtype: object

In [41]:
flats['Всего этажей'] = flats['Этаж'].str.extract(r'из (.*)').fillna(0).astype(int)
flats['Этаж'] = flats['Этаж'].str.extract(r'(.*?) из').fillna(0).astype(int)
flats[['Всего этажей','Этаж']].head()

,Всего этажей,Этаж
0,12,9
1,6,1
2,9,3
3,12,6
4,8,8


In [42]:
flats['Санузел'].head()

0    совмещённый
1    совмещённый
2            NaN
3     раздельный
4     раздельный
Name: Санузел, dtype: object

In [43]:
flats['Санузел'] = flats['Санузел'].fillna('')
flats['Санузел'].value_counts()

Санузел
раздельный     4019
               2593
совмещённый    2581
Name: count, dtype: int64

In [44]:
flats['Кол-во санузлов'].head()

0    1.0
1    1.0
2    NaN
3    2.0
4    1.0
Name: Кол-во санузлов, dtype: float64

In [45]:
flats['Кол-во санузлов'] = flats['Кол-во санузлов'].fillna('Неизвестно')
flats['Кол-во санузлов'].value_counts()

Кол-во санузлов
1.0           6488
Неизвестно    2284
2.0            318
3.0             94
4.0              8
5.0              1
Name: count, dtype: int64

In [46]:
flats['Балкон/лоджия'].head()

0    есть
1     NaN
2    есть
3    есть
4    есть
Name: Балкон/лоджия, dtype: object

In [47]:
flats['Балкон/лоджия'] = flats['Балкон/лоджия'].fillna('нет')
flats['Балкон/лоджия'].value_counts()

Балкон/лоджия
есть    6349
нет     2844
Name: count, dtype: int64

In [48]:
flats['Вид из окна'].head()

0    на улицу
1     во двор
2         NaN
3    на улицу
4    на улицу
Name: Вид из окна, dtype: object

In [49]:
flats['Вид из окна'] = flats['Вид из окна'].fillna('Неизвестно')
flats['Вид из окна'].value_counts()

Вид из окна
Неизвестно    5274
во двор       3097
на улицу       822
Name: count, dtype: int64

In [50]:
flats[['Срок сдачи', 'Год постройки']].head()

,Срок сдачи,Год постройки
0,3 кв. 2023 г.,NaN
1,NaN,2018.0
2,2 кв. 2025 г.,NaN
3,NaN,NaN
4,NaN,2023.0


In [51]:
flats['Срок сдачи'] = flats['Срок сдачи'].fillna('')
def year(row):
    y = 0
    if row['Год постройки']==row['Год постройки']:
        y = row['Год постройки']
    elif (row['Срок сдачи']==row['Срок сдачи']) & (row['Срок сдачи']!=''):
        p = row['Срок сдачи'].find(' г.')
        if p>0:
            y = row['Срок сдачи'][p-4:p]
    return int(y)
flats['Год постройки'] = flats.apply(year,axis=1)
flats[['Срок сдачи', 'Год постройки']].head()

,Срок сдачи,Год постройки
0,3 кв. 2023 г.,2023
1,,2018
2,2 кв. 2025 г.,2025
3,,0
4,,2023


In [52]:
flats['Высота потолков'].head(10)

0    2,75 м
1       3 м
2    2,75 м
3     2,8 м
4     3,3 м
5       NaN
6       NaN
7    2,77 м
8    2,76 м
9     2,8 м
Name: Высота потолков, dtype: object

In [53]:
flats['Высота потолков'] = flats['Высота потолков'].str.replace(' м', '').str.replace(',','.').fillna(0).astype(float)
flats['Высота потолков'].head(10)

0    2.75
1    3.00
2    2.75
3    2.80
4    3.30
5    0.00
6    0.00
7    2.77
8    2.76
9    2.80
Name: Высота потолков, dtype: float64

In [54]:
flats['Лифт'].head(10)

0    есть
1    есть
2    есть
3    есть
4    есть
5     NaN
6    есть
7    есть
8    есть
9    есть
Name: Лифт, dtype: object

In [55]:
flats['Лифт'] = flats['Лифт'].fillna('нет')
flats['Лифт'].head(10)

0    есть
1    есть
2    есть
3    есть
4    есть
5     нет
6    есть
7    есть
8    есть
9    есть
Name: Лифт, dtype: object

In [56]:
flats['Парковка'].head()

0    закрытая
1         NaN
2    закрытая
3         NaN
4         NaN
Name: Парковка, dtype: object

In [57]:
flats['Парковка'] = flats['Парковка'].fillna('Неизвестно')
flats['Парковка'].value_counts()

Парковка
закрытая      4592
Неизвестно    4588
открытая        10
подземная        3
Name: count, dtype: int64

In [58]:
flats['Территория'].head()

0    закрытая
1    закрытая
2         NaN
3    закрытая
4    закрытая
Name: Территория, dtype: object

In [59]:
flats['Территория'] = flats['Территория'].fillna('Неизвестно')
flats['Территория'].value_counts()

Территория
Неизвестно    7192
закрытая      2001
Name: count, dtype: int64

In [60]:
flats['Тип жилья'].head()

0            NaN
1    апартаменты
2            NaN
3            NaN
4            NaN
Name: Тип жилья, dtype: object

In [61]:
flats['Тип жилья'] = flats['Тип жилья'].fillna('не апартаменты')
flats['Тип жилья'].value_counts()

Тип жилья
не апартаменты    9018
апартаменты        175
Name: count, dtype: int64

In [62]:
flats['Ремонт'].head()

0                 NaN
1        Дизайнерский
2                 NaN
3                Евро
4    Черновая отделка
Name: Ремонт, dtype: object

In [63]:
flats['Ремонт'] = flats['Ремонт'].fillna('Неизвестно')
flats['Ремонт'].value_counts()

Ремонт
Неизвестно          7679
Черновая отделка     648
Евро                 349
Дизайнерский         174
Чистовая отделка     171
Нужен ремонт         131
Хороший               41
Name: count, dtype: int64

In [64]:
# наличие всяких штук не буду смотреть отдельно, они или есть, или нет
flats['Кондиционер'] = flats['Кондиционер'].fillna('нет')
flats['Посудомойка'] = flats['Посудомойка'].fillna('нет')
flats['Холодильник'] = flats['Холодильник'].fillna('нет')
flats['Мебель'] = flats['Мебель'].fillna('нет')
flats['Интернет'] = flats['Интернет'].fillna('нет')
flats['Кухонная мебель'] = flats['Кухонная мебель'].fillna('нет')
flats['Телевизор'] = flats['Телевизор'].fillna('нет')
flats['Стиральная машина'] = flats['Стиральная машина'].fillna('нет')
flats['Консьерж'] = flats['Консьерж'].fillna('нет')

In [65]:
# еще всякие штуки, где если Nan - это неизвестно
flats['Материал стен'] = flats['Материал стен'].fillna('Неизвестно')
flats['Отопление'] = flats['Отопление'].fillna('Неизвестно')
flats['Вид из окон'] = flats['Вид из окон'].fillna('Неизвестно')
flats['Серия дома'] = flats['Серия дома'].fillna('Неизвестно')
flats['Временная эпоха'] = flats['Временная эпоха'].fillna('Неизвестно')

In [66]:
flats.to_csv('data/clean_data_from_metr2_2023.csv',sep=';')

<h2>Смотрим рост цен на недвижку всего и по районам</h2>

In [67]:
import pandas as pd
flats_old = pd.read_csv('data/clean_data_from_cian_2021.csv',sep=';')

In [68]:
flats_old.head()

,link,price,total_area,living_area,kitchen_area,floor,type,height,bathrooms,balconies,...,renovation,view,rooms,city,area,neighborhood,street,metro_name,metro_km,number_of_floors
0,https://spb.cian.ru/sale/flat/234775065/,13143249.0,74.13,26.3,22.0,3,Новостройка,3.3,2.0,1.0,...,NaN,NaN,2,Санкт-Петербург,р-н Приморский,Юнтолово,"Планерная ул., 94",Комендантский проспект,1.98,12
1,https://spb.cian.ru/sale/flat/239273301/,12430600.0,47.81,NaN,NaN,2,Новостройка,NaN,0.0,0.0,...,NaN,NaN,1,Санкт-Петербург,р-н Петроградский,Посадский,"ул. Рентгена, 25",Петроградская,1.04,8
2,https://spb.cian.ru/sale/flat/250966190/,13800000.0,44.10,14.1,19.4,2,Новостройка,NaN,0.0,0.0,...,NaN,NaN,1,Санкт-Петербург,р-н Курортный,мкр. Сестрорецк,"ул. Максима Горького, 2Ас2",Беговая,9.57,5
3,https://spb.cian.ru/sale/flat/249950664/,8730851.0,56.29,29.3,10.9,5,Новостройка,NaN,2.0,2.0,...,NaN,NaN,2,Санкт-Петербург,р-н Приморский,Юнтолово,Нью Тайм жилой комплекс,Комендантский проспект,1.98,13
4,https://spb.cian.ru/sale/flat/250766812/,13650000.0,70.80,NaN,10.0,13,Вторичка,2.8,1.0,1.0,...,Евроремонт,На улицу и двор,2,Санкт-Петербург,р-н Приморский,Комендантский аэродром,"аллея Поликарпова, 6к1",Пионерская,1.04,19


In [69]:
flats_old['metr_price'] = flats_old['price']/flats_old['total_area']

In [70]:
means2021 = flats_old['metr_price'].describe()
means2021.map('{:.2f}'.format)

count       1414.00
mean      208232.99
std       110679.50
min        68788.50
25%       145675.68
50%       172714.61
75%       234145.80
max      1400000.00
Name: metr_price, dtype: object

In [71]:
flats_new = pd.read_csv('data/clean_data_from_metr2_2023.csv',sep=';')

In [72]:
means2023 = flats_new['Цена за метр'].describe()
means2023.map('{:.2f}'.format)

count       9193.00
mean      218625.98
std        71418.83
min        73033.71
25%       174743.22
50%       203703.70
75%       252325.58
max      1709558.82
Name: Цена за метр, dtype: object

In [73]:
# на сколько в среднем выросла недвижка за 2 года в %
(means2023['mean'] - means2021['mean']) / means2021['mean'] *100

4.991039734912574

In [74]:
# Теперь посмотрим первичку и вторичку
means2021_1 = flats_old[flats_old['type']=='Новостройка']['metr_price'].describe()
means2021_1.map('{:.2f}'.format)

count       404.00
mean     198107.08
std       94904.87
min       84477.61
25%      145867.64
50%      169752.03
75%      222455.40
max      815850.82
Name: metr_price, dtype: object

In [75]:
means2021_2 = flats_old[flats_old['type']=='Вторичка']['metr_price'].describe()
means2021_2.map('{:.2f}'.format)

count       1010.00
mean      212283.35
std       116189.04
min        68788.50
25%       145675.68
50%       174083.41
75%       242519.20
max      1400000.00
Name: metr_price, dtype: object

In [76]:
means2023_1 = flats_new[flats_new['Срок сдачи'].notna()]['Цена за метр'].describe()
means2023_1.map('{:.2f}'.format)

count      3889.00
mean     219266.06
std       51694.63
min      105420.79
25%      182379.39
50%      210284.82
75%      256338.00
max      469750.00
Name: Цена за метр, dtype: object

In [77]:
means2023_2 = flats_new[flats_new['Срок сдачи'].isna()]['Цена за метр'].describe()
means2023_2.map('{:.2f}'.format)

count       5304.00
mean      218156.66
std        82954.55
min        73033.71
25%       169847.33
50%       199261.23
75%       248041.32
max      1709558.82
Name: Цена за метр, dtype: object

In [78]:
print('Цена на первичку поднялись на ', (means2023_1['mean'] - means2021_1['mean']) / means2021_1['mean'] *100, '%' )
print('Цена на вторичку поднялись на ', (means2023_2['mean'] - means2021_2['mean']) / means2021_2['mean'] *100, '%' )

Цена на первичку поднялись на  10.680577080442614 %
Цена на вторичку поднялись на  2.766730745335023 %


In [79]:
# Посмотрим по районам
mean_area = flats_old.groupby(['area'])['metr_price'].mean().reset_index()
mean_area['area'] = mean_area['area'].str.replace('р-н ','')
mean_area.rename(columns = {'metr_price':'metr_price2021'}, inplace=True)
mean_area1 = flats_new.groupby(['Район'])['Цена за метр'].mean().reset_index()
mean_area1['Район'] = mean_area1['Район'].str.replace(' район','')
mean_area1.rename(columns = {'Район':'area', 'Цена за метр':'metr_price2023'}, inplace=True)
mean_area = mean_area.merge(mean_area1, on=['area'], how='left')
mean_area['diff, %'] = (mean_area['metr_price2023'] - mean_area['metr_price2021'])/ mean_area['metr_price2021']*100
mean_area

,area,metr_price2021,metr_price2023,"diff, %"
0,Адмиралтейский,219589.932183,273738.682395,24.659031
1,Василеостровский,206045.477773,263126.991657,27.703357
2,Выборгский,184313.622135,200157.889813,8.596363
3,Калининский,157053.503653,224985.301817,43.253921
4,Кировский,150048.900268,171585.234998,14.352877
5,Колпинский,107269.490509,140552.748600,31.027702
6,Красногвардейский,154624.312267,211571.891503,36.829641
7,Красносельский,144509.116603,180052.592545,24.596009
8,Кронштадтский,120876.524994,153826.956774,27.259579
9,Курортный,243061.285793,257840.786951,6.080566


<h2>Цены на недвижку по домам</h2>

Что нужно сделать
- Взять датасеты из 2021 и 2023
- Оставить адреса, площадь, кол-во комнат, тип (новостройка или вторичка) и цены
- Почистить адреса и привести датасеты к единому виду
- Посчитать среднюю цену за метр в доме
- Соединить датасеты, посмотреть рост по домам
- Посмотреть дома, которые были новостройками, а стали вторичками

In [11]:
import pandas as pd
flats_old = pd.read_csv('data/clean_data_from_cian_2021.csv',sep=';')
flats_new = pd.read_csv('data/clean_data_from_metr2_2023.csv', sep=';')

In [12]:
flats_old = flats_old[[ 'total_area', 'price', 'rooms', 'type', 'area', 'street']]
flats_new = flats_new[['Площадь квартиры', 'Цена', 'Комнат', 'Срок сдачи', 'Район', 'Улица', 'Номер дома']]
flats_new.rename(columns={'Площадь квартиры':'total_area', 'Цена':'price', 'Комнат':'rooms',
                          'Срок сдачи': 'type','Район':'area', 'Улица':'street', 'Номер дома':'house'}, inplace=True)

In [13]:
# причешем старый датасет
def addr_old(row):
    names = [('ул.', 'улица'),
             ('аллея','аллея'),
             ('просп.','проспект'),
             ('наб.','набережная'),
             ('линия','линия'),
             ('пер.','переулок'),
             ('дор.','дорога'),
             ('ш.','шоссе'),
             ('бул.','бульвар'),
             ('проезд', 'проезд'),
             ('пл.', 'площадь')]
    if row['street'].find(',') >= 0:
        row['house'] = row['street'][row['street'].rindex(',') + 2:].strip()
        row['street'] = row['street'][:row['street'].rindex(',')]
        if row['street'].find(',') >=0 :
            row['street'] = row['street'][row['street'].rindex(',') + 2:]
    for name in names:
        if row['street'].find(name[0]) >= 0:
            row['street_name'] = name[1]
            row['street'] = row['street'].replace(name[0], '').strip()
            break
    row['area'] = row['area'].replace('р-н ','').strip()
    return row
flats_old['house'] = ''
flats_old['street_name'] = ''
flats_old['street'] = flats_old['street'].astype(str)
flats_old = flats_old.apply(addr_old, axis=1)
flats_old = flats_old[(flats_old['street_name'] != '') & (flats_old['house'] != '')]
flats_old

,total_area,price,rooms,type,area,street,house,street_name
0,74.13,13143249.0,2,Новостройка,Приморский,Планерная,94,улица
1,47.81,12430600.0,1,Новостройка,Петроградский,Рентгена,25,улица
2,44.10,13800000.0,1,Новостройка,Курортный,Максима Горького,2Ас2,улица
4,70.80,13650000.0,2,Вторичка,Приморский,Поликарпова,6к1,аллея
6,63.57,12300000.0,2,Новостройка,Выборгский,Александра Матросова,8к1,улица
...,...,...,...,...,...,...,...,...
1408,57.80,8930100.0,2,Новостройка,Невский,Дыбенко,7к1,улица
1409,55.02,6766360.0,2,Новостройка,Приморский,Ивинская,9,улица
1410,54.87,6790163.0,2,Новостройка,Приморский,Ивинская,9,улица
1411,31.60,7500000.0,1,Вторичка,Приморский,Оптиков,37,улица


In [14]:
# Причешем новый датасет
def addr_new(row):
    names = ['улица', 'аллея', ' проспект', 'проспект', 'набережная', ' линия', 'переулок', 'дорога', 'шоссе', 'бульвар', 'проезд', 'площадь', 'коса', 'проток']
    if row['type'] == row['type']:
        row['type'] = 'Новостройка'
    else:
        row['type'] = 'Вторичка'
    row['area'] = row['area'].replace('район','').strip()
    row['house'] = row['house'].strip()
    for name in names:
        if row['street'].find(name) >= 0:
            row['street_name'] = name
            row['street'] = row['street'].replace(name, '').strip()
            break
    return row
flats_new = flats_new[(flats_new['street'].notna()) & (flats_new['house'].notna())]
flats_new['street_name'] = ''
flats_new = flats_new.apply(addr_new, axis=1)
flats_new = flats_new[flats_new['street_name'] != '']
flats_new

,total_area,price,rooms,type,area,street,house,street_name
0,34.0,10500000.0,1,Новостройка,Выборгский,Большой Сампсониевский,7,проспект
1,53.3,32700000.0,1,Вторичка,Петроградский,Петровский,2с2,проспект
3,67.4,12500000.0,2,Вторичка,Красносельский,Ленинский,72к1,проспект
4,63.1,17490000.0,2,Вторичка,Московский,Черниговская,11к1,улица
5,33.9,11319999.0,1,Новостройка,Красногвардейский,Заневский,65,проспект
...,...,...,...,...,...,...,...,...
9188,17.5,6500000.0,0,Вторичка,Адмиралтейский,Спасский,6-8,переулок
9189,34.0,6400000.0,1,Вторичка,Фрунзенский,Бухарестская,72к1,улица
9190,30.2,6447519.0,0,Вторичка,Выборгский,Ольгинская,3/20,дорога
9191,33.1,8400000.0,0,Вторичка,Приморский,Богатырский,2А,проспект


In [18]:
# Сгруппируем все данные по старому датасету
group_old = flats_old.groupby(['area','street','street_name','house', 'type']).agg({'total_area':'sum', 'price':'sum'})
group_old['metr_price_2021'] = group_old['price'] / group_old['total_area']
group_old = group_old.reset_index()
group_old

,area,street,street_name,house,type,total_area,price,metr_price_2021
0,Адмиралтейский,10-я Красноармейская,улица,6,Вторичка,61.4,9900000.0,161237.785016
1,Адмиралтейский,11-я Красноармейская,улица,11с2,Новостройка,285.5,75693032.0,265124.455342
2,Адмиралтейский,11-я Красноармейская,улица,11с3,Новостройка,81.2,18880963.0,232524.174877
3,Адмиралтейский,11-я Красноармейская,улица,15А,Вторичка,53.7,14217054.0,264749.608939
4,Адмиралтейский,12-я Красноармейская,улица,26Ж,Новостройка,78.3,18970587.0,242280.804598
...,...,...,...,...,...,...,...,...
893,Центральный,Чайковского,улица,1,Вторичка,100.4,24500000.0,244023.904382
894,Центральный,Шпалерная,улица,3,Вторичка,32.9,5250000.0,159574.468085
895,Центральный,Шпалерная,улица,42,Вторичка,113.4,31000000.0,273368.606702
896,Центральный,Шпалерная,улица,50Б,Вторичка,159.6,65000000.0,407268.170426


In [19]:
# Сгруппируем все данные по новому датасету
group_new = flats_new.groupby(['area','street','street_name','house', 'type']).agg({'total_area':'sum', 'price':'sum'})
group_new['metr_price_2023'] = group_new['price'] / group_new['total_area']
group_new = group_new.reset_index()
group_new

,area,street,street_name,house,type,total_area,price,metr_price_2023
0,Адмиралтейский,10-я Красноармейская,улица,9,Вторичка,26.0,7499999.0,288461.500000
1,Адмиралтейский,11-я Красноармейская,улица,11,Вторичка,106.0,34631500.0,326712.264151
2,Адмиралтейский,11-я Красноармейская,улица,11с3,Вторичка,718.2,244380000.0,340267.335004
3,Адмиралтейский,11-я Красноармейская,улица,13,Вторичка,61.8,10400000.0,168284.789644
4,Адмиралтейский,11-я Красноармейская,улица,14,Вторичка,67.5,11200000.0,165925.925926
...,...,...,...,...,...,...,...,...
2916,Центральный,реки Мойки,набережная,24,Вторичка,104.0,38000000.0,365384.615385
2917,Центральный,реки Фонтанки,набережная,54,Вторичка,286.0,187728800.0,656394.405594
2918,Центральный,реки Фонтанки,набережная,66,Вторичка,120.1,37000000.0,308076.602831
2919,Центральный,реки Фонтанки,набережная,75,Вторичка,94.2,17900000.0,190021.231423


In [20]:
# Соединим два датасета и посмотрим на рост цен по домам
group = group_old.merge(group_new, on=['area','street','street_name','house'], how='inner', suffixes=('_old', '_new'))
group['diff %'] = (group['metr_price_2023'] - group['metr_price_2021']) / group['metr_price_2021'] * 100
group

,area,street,street_name,house,type_old,total_area_old,price_old,metr_price_2021,type_new,total_area_new,price_new,metr_price_2023,diff %
0,Адмиралтейский,11-я Красноармейская,улица,11с3,Новостройка,81.2,18880963.0,232524.174877,Вторичка,718.2,244380000.0,340267.335004,46.336326
1,Адмиралтейский,Спасский,переулок,6-8,Вторичка,40.0,7249000.0,181225.000000,Вторичка,41.0,13350000.0,325609.756098,79.671544
2,Василеостровский,Вадима Шефнера,улица,10к1,Вторичка,23.0,4600000.0,200000.000000,Вторичка,50.0,12500000.0,250000.000000,25.000000
3,Василеостровский,Вадима Шефнера,улица,12к1,Вторичка,44.0,7490000.0,170227.272727,Вторичка,120.4,27800000.0,230897.009967,35.640433
4,Василеостровский,Вадима Шефнера,улица,14к1,Вторичка,40.5,6950000.0,171604.938272,Вторичка,74.1,15250000.0,205802.968961,19.928349
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Центральный,Рубинштейна,улица,11,Вторичка,80.0,34990000.0,437375.000000,Вторичка,126.7,33000000.0,260457.774270,-40.449780
192,Центральный,Рылеева,улица,23,Вторичка,21.2,3720000.0,175471.698113,Вторичка,15.9,3850000.0,242138.364780,37.992832
193,Центральный,Синопская,набережная,32/35,Вторичка,41.2,6620000.0,160679.611650,Вторичка,131.0,20900000.0,159541.984733,-0.708010
194,Центральный,Херсонская,улица,43/12,Вторичка,120.0,26257942.0,218816.183333,Вторичка,29.2,11500000.0,393835.616438,79.984684


In [23]:
# посмотрим новостройки, которые стали вторичками
diff_type = group[(group['type_old'] == 'Новостройка') & (group['type_new'] == 'Вторичка')]
metr_price_old = diff_type['price_old'].sum()/diff_type['total_area_old'].sum()
print('Средняя цена в 2021:', metr_price_old)
metr_price_new = diff_type['price_new'].sum()/diff_type['total_area_new'].sum()
print('Средняя цена в 2023:', metr_price_new)
print('Разница:', (metr_price_new - metr_price_old)/metr_price_old*100, '%')


Средняя цена в 2021: 254267.89123506716
Средняя цена в 2023: 290555.6814700489
Разница: 14.27147960315374 %


In [24]:
diff_type

,area,street,street_name,house,type_old,total_area_old,price_old,metr_price_2021,type_new,total_area_new,price_new,metr_price_2023,diff %
0,Адмиралтейский,11-я Красноармейская,улица,11с3,Новостройка,81.20,18880963.0,232524.174877,Вторичка,718.2,2.443800e+08,340267.335004,46.336326
5,Василеостровский,Вадима Шефнера,улица,4,Новостройка,68.60,11233400.0,163752.186589,Вторичка,139.4,2.924900e+07,209820.659971,28.133043
10,Василеостровский,Челюскина,улица,2,Новостройка,66.50,12500000.0,187969.924812,Вторичка,32.0,1.110000e+07,346875.000000,84.537500
12,Выборгский,Александра Матросова,улица,8к1,Новостройка,175.57,35390000.0,201572.022555,Вторичка,418.8,1.146800e+08,273829.990449,35.847221
31,Кировский,Двинская,улица,6,Новостройка,213.61,39197600.0,183500.772436,Вторичка,106.6,2.765000e+07,259380.863039,41.351374
39,Красногвардейский,Чарушинская,улица,22к1,Новостройка,51.50,6968000.0,135300.970874,Вторичка,63.3,1.069000e+07,168878.357030,24.816811
56,Курортный,Максима Горького,улица,2Ас3,Новостройка,57.00,16500000.0,289473.684211,Вторичка,45.0,1.860000e+07,413333.333333,42.787879
59,Московский,1-й Предпортовый,проезд,13,Новостройка,25.31,5050000.0,199525.879099,Вторичка,36.3,8.850000e+06,243801.652893,22.190492
60,Московский,1-й Предпортовый,проезд,15,Новостройка,48.32,8739000.0,180856.788079,Вторичка,111.8,3.330000e+07,297853.309481,64.690147
68,Московский,Малая Митрофаньевская,улица,8к1,Новостройка,108.20,15909790.0,147040.573013,Вторичка,110.1,2.350000e+07,213442.325159,45.158796


<h2>Заключение</h2>

Ипотека
- По заданным параметрам ипотеки - лучше ее досрочно гасить сразу с первого месяца. Если начать через три месяца - переплата будет больше 100 тыщ.
- Если вкидывать в ипотеку по 200 тыщ ежемесячно, то платить надо будет 58 месяцев (последний платеж в мае 2028), переплата 1 150 763 р.
- Если вкидывать в ипотеку по 250 тыщ ежемесячно, то платить надо будет 46 месяцев (последний платеж в мае 2027), переплата 897 950 р.

Общий рост недвижки
Для дальнейших сравнений взяты данные с циана лета 2021 года (около 1,5к объявлений) и метра квадратного лета 2023 года (около 10к объявлений).
- Средний рост цены за квадратный метр за 2 года - около 5%. Цена на первичку поднялись на 10.7 %. Цена на вторичку поднялись на 2.8 %.
- Больше всего недвижка подорожала в Петродворцовом районе - 52 %, отрицательный рекорд - Центральный район, там спад на 0,4%
- Интересующие нас районы - Выборгский и Петроградский. Там рост 8,6% и 19,7% соответственно. Я бы больше ориентировалась на Петроградку. В любом случае рост цены вероятнее всего окупит годовой процент ипотеки.
- Рост недвижки, которая была вторичкой, а стала первичкой - 14,27%.

Самый впечатляющий рост у необчных ЖК (типа комфорт и бизнес-класс).
- ЖК на Ждановской 45 (ЖК Леонтьевский мыс) на Петровской косе, элитная илитность, рост 124 %
- Нижне-Каменская 7к1, Коменда но малоэтажный ЖК с видом на лес, рост 120 %
- Челюскина 2, дом на намыве с золотым конусом-шпилем, позиционируется на бизнес-класс, рост 85%
- Земледельческая, 3, ЖК Терра класса бизнес-лайт (хз что это такое), около станции ланской с видом на парк, рост 71%


